<a href="https://colab.research.google.com/github/TomoharuKurosu/TomoharuKurosu/blob/main/keiomacro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
#授業
import numpy as np
from scipy.stats import norm

def tauchen(n, mu, rho, sigma):

    m = 1.0 / np.sqrt(1.0-rho**2)

    state_space = np.linspace(mu-m*sigma, mu+m*sigma, n)

    d = (state_space[n-1] - state_space[0]) / (n-1)


    transition_matrix = np.zeros((n, n))

    for i in range(n):
        for j in range(n):
            if j == 0:
                transition_matrix[i, j] = norm.cdf((state_space[j] + d/2 - rho*state_space[i])/sigma)
            elif j == n-1:
                transition_matrix[i, j] = 1 - norm.cdf((state_space[j] - d/2 - rho*state_space[i])/sigma)
            else:
                z_high = (state_space[j] - rho*state_space[i] + d/2) / sigma
                z_low = (state_space[j] - rho*state_space[i] - d/2) / sigma
                transition_matrix[i, j] = norm.cdf(z_high) - norm.cdf(z_low)

    return transition_matrix, state_space


In [6]:
NH = 2
NA = 401
a_l = 0.0
a_u = 20.0



rho = 0.6
alpha = 0.25
delta = 0.03
sigma = 1.50
beta = 0.98

sigma_eps = np.sqrt(0.6*(1-rho**2))

pi, h = tauchen(NH, -0.7, rho, sigma_eps)

h = np.exp(h)



probst = np.ones((NH))/NH

probst_new = np.zeros((NH))

test = 10

while test >0.0000001:
    probst_new = np.zeros((NH))
    for j in range(NH):
        for jplus in range(NH):
            probst_new[jplus] += pi[j, jplus]*probst[j]
    test = np.max(np.abs(probst_new - probst))
    probst = probst_new

HH = np.sum(h*probst)



In [ ]:
KK = 10.0

w = (1-alpha)*KK**(alpha) * HH**(-alpha)
r = alpha*KK**(alpha-1) * HH**(1-alpha) - delta

a = np.linspace(a_l, a_u, NA)

util = np.full((NA, NA, NH), -10000.0)

for ia in range(NA):
    for ih in range(NH):
        for iap in range(NA):
            cons = w*h[ih] + (1.0 + r)*a[ia] - a[iap]
            if cons>0:
                util[iap, ia, ih] = cons**(1.0-sigma)/(1.0-sigma)

v = np.zeros((NA, NH))
v_new = np.zeros((NA, NH))
iaplus = np.full((NA, NH), -10000)
iaplus_new = np.full((NA, NH), -10000)

test = 10
reward = np.zeros((NA, NA, NH))

while test != 0:
    for ia in range(NA):
        for ih in range(NH):
            reward[:, ia, ih] = util[:, ia, ih]
            for ihp in range(NH):
                reward[:, ia, ih] += beta*pi[ih, ihp]*v[:, ihp]

    for ia in range(NA):
        for ih in range(NH):
            v_new[ia, ih] = np.max(reward[:, ia, ih])
            iaplus_new[ia, ih] = np.argmax(reward[:, ia, ih])

    test = np.max(iaplus_new - iaplus)
    v = v_new
    iaplus = iaplus_new.copy()

aplus = a[iaplus]


In [ ]:
util[:, 1, 1]